<p>
Prepared by Vytautas Bielinskas. 2020. <br>
Download data from: https://finance.yahoo.com/quote/GOOG/history/
</p>

<h2>PART 1. Data Pre-processing</h2>

<h3>Step #0. Fire the system</h3>

In [ ]:
# Import modules and packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
import plotly.graph_objects as go # for visualization
import os
import sys 
sys.path.append("../../Function/")
import function 

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

%matplotlib inline

<h3>Step #1. Read data</h3>

In [ ]:
# Importing Training Set
df = pd.read_excel('../../../site_info_ver_4_lite.xlsx')


In [ ]:
dataset = df.copy()
dataset = dataset[dataset.Site_Name==dataset.Site_Name.unique()[2]]
site_name = dataset.Site_Name.unique()[0]
print(site_name)
# dataset_train.set_index("Date",inplace=True)
dataset[['so2','nox','pm10','pm2.5', 'is_Site_exceeded_nox', 'is_Site_exceeded_so2', 'is_Site_exceeded_pm10', 'is_Site_exceeded_pm2.5']].isna().sum()
dataset.drop(['so2','nox','pm10','pm2.5','is_Site_exceeded_nox', 'is_Site_exceeded_so2', 'is_Site_exceeded_pm10', 'is_Site_exceeded_pm2.5','Day_before_Total','Tourists_Count', 'Total'],axis=1,inplace=True)
dataset = function.remove_unique_one(dataset)
dataset = function.remove_high_corr(dataset, 'Israelis_Count')
dataset = function.split_date(dataset)
print('dataset shape before remove outliers',dataset.shape)
dataset = function.remove_outliers(dataset, 'Israelis_Count')
tmp = dataset.Israelis_Count
dataset.drop('Israelis_Count',axis=1,inplace=True)
print('dataset shape after remove outliers',dataset.shape)
dataset['Israelis_Count']=tmp
# dataset.set_index('Date',inplace=True)
print()
print(dataset.columns)
dataset.head(1)

In [ ]:
n_predict = 1   # Number of days we want top predict into the future
n_past = 2   # Number of past days we want to use to predict the future
cols = dataset.drop("Date",axis=1).columns
target='Israelis_Count'
target_title='Israelis'
def X_y_data(dataset):
    # Select features (columns) to be involved intro training and predictions

    # Extract dates (will be used in visualization)
    datelist = list(dataset['Date'])
    
    # datelist = [dt.datetime.strptime(date, '%Y-%m-%d').date() for date in datelist]
    dataset=dataset.set_index('Date')
    print('set shape == {}'.format(dataset.shape))
    print('All timestamps == {}'.format(len(datelist)))
    print('Featured selected: {}'.format(cols))
    dataset = dataset[cols].astype(str)
    for i in cols:
        for j in dataset.index:
            dataset[i][j] = dataset[i][j].replace(',', '')

    dataset = dataset.astype(float)

    # Using multiple features (predictors)
    training_set = dataset.values

    print('Shape of set == {}.'.format(training_set.shape))
    print(training_set)

    #first get log from visitors : 
    
    a = training_set[:,:-1]
    b = np.log(training_set[:,-1]+0.001)
    print('a',a.shape,a)
    print('b',b.shape,b)
    # np.reshape(b, b.shape[0],)
    training_set = np.insert(a, len(cols)-1,values=b, axis=1) # Insert values before column 3

    print('training_set\n',training_set)
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler

    sc = StandardScaler()
    training_set_scaled = sc.fit_transform(training_set)

    sc_predict = StandardScaler()
    sc_predict.fit_transform(training_set[:, -1:])
    # Creating a data structure with 90 timestamps and 1 output
    X_train = []
    y_train = []



    for i in range(n_past, len(training_set_scaled) - n_predict +1):
        X_train.append(training_set_scaled[i - n_past:i,:])
        y_train.append(training_set_scaled[i + n_predict - 1:i + n_predict,-1])
    print()

    return dataset,datelist,np.array(X_train), np.array(y_train),sc_predict
size = round(len(dataset)*0.85)

dataset_train = dataset[:size]
dataset_test = dataset[size:]
print('Training print')
print()
dataset_train,datelist_train,X_train, y_train,sc_predict_train = X_y_data(dataset_train)

print('Testing print')
print()
dataset_test,datelist_test,X_test, y_test,sc_predict_test = X_y_data(dataset_test)

print('X_train shape == {}.'.format(X_train.shape))
print('y_train shape == {}.'.format(y_train.shape))
print('X_test shape == {}.'.format(X_test.shape))
print('y_test shape == {}.'.format(y_test.shape))


<h2>PART 2. Create a model. Training</h2>

<h3>Step #3. Building the LSTM based Neural Network</h3>

In [ ]:
# Import Libraries and packages from Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
# from keras.optimizers import ADAM

In [ ]:
from keras import backend
 
def rmse(y_true, y_pred):
	return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

# Initializing the Neural Network based on LSTM
model = Sequential()

# Adding 1st LSTM layer
model.add(LSTM(units=128, return_sequences=True, input_shape=(n_past, dataset_train.shape[1])))
# Adding Dropout
model.add(Dropout(0.25))
# Adding 2nd LSTM layer
model.add(LSTM(units=128, return_sequences=False))

#Adding Dropout
model.add(Dropout(0.25))
# # Adding 2nd LSTM layer
# model.add(LSTM(units=64, return_sequences=True))

# #Adding Dropout
# model.add(Dropout(0.15))
# # Adding 2nd LSTM layer
# model.add(LSTM(units=64, return_sequences=True))

# #Adding Dropout
# model.add(Dropout(0.15))
# model.add(LSTM(units=32, return_sequences=False))

# #Adding Dropout
# model.add(Dropout(0.15))

# Output layer
model.add(Dense(units=1))

# Compiling the Neural Network
model.compile(optimizer = 'adam', loss=[rmse],metrics='mean_squared_error')

<h3>Step #4. Start training</h3>

In [ ]:
%%time
es = EarlyStopping(monitor='val_loss', min_delta=1e-10,verbose=1, patience=20)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,verbose=1, patience=20)
mcp = ModelCheckpoint(filepath='weights.h5', monitor='val_loss',verbose=1 ,save_best_only=True, save_weights_only=True)

tb = TensorBoard('logs')

history = model.fit(X_train, y_train, shuffle=True, epochs=1000, callbacks=[es, rlr, mcp, tb], validation_split=0.2, verbose=1, batch_size=1)

<p>
Notes:<br>
<ul>
<li><b>EarlyStopping</b> - Stop training when a monitored metric has stopped improving.</li>
<li><code>monitor</code> - quantity to be monitored.</li>
<li><code>min_delta</code> - minimum change in the monitored quantity to qualify as an improvement, i.e. an absolute change of less than <code>min_delta</code>, will count as no improvement.</li>
<li><code>patience</code> - number of epochs with no improvement after which training will be stopped.</li>
</ul>

<ul>
<li><b>ReduceLROnPlateau</b> - Reduce learning rate when a metric has stopped improving.</li>
<li><code>factor</code> - factor by which the learning rate will be reduced. <code>new_lr = lr * factor</code>.</li>
</ul>
</p>

<hr>

<p>
The last date for our training set is <code>30-Dec-2016</code>.<br>
</p>

<p>
We will perform predictions for the next <b>20</b> days, since <b>2017-01-01</b> to <b>2017-01-20</b>.
</p>

<h2>PART 3. Make future predictions</h2>

In [ ]:
# # Generate list of sequence of days for predictions
# datelist_future = pd.date_range(datelist_train[-1], periods=n_predict, freq='1d').tolist()
# datelist_future


In [ ]:
'''
Remeber, we have datelist_train from begining.
'''

# Convert Pandas Timestamp to Datetime object (for transformation) --> FUTURE
datelist_future_ = []
for this_timestamp in datelist_test:
    datelist_future_.append(this_timestamp.date())
# datelist_future_

<h3>Step #5. Make predictions for future dates</h3>

In [ ]:
# Perform predictions
model.load_weights('weights.h5')
predictions_future = model.predict(X_test)
len(predictions_future)
# predictions_train = model.predict(X_train[n_past:])

In [ ]:
# Inverse the predictions to original measurements

# ---> Special function: convert <datetime.date> to <Timestamp>
def datetime_to_timestamp(x):
    '''
        x : a given datetime value (datetime.date)
    '''
    return datetime.strptime(x.strftime('%Y%m%d'), '%Y%m%d')


y_pred_future = np.exp(sc_predict_test.inverse_transform(predictions_future))
print(predictions_future.shape)


PREDICTIONS_FUTURE = pd.DataFrame(y_pred_future, columns=[target]).set_index(pd.Series(datelist_test[n_past:]))


PREDICTIONS_FUTURE.head(3)

<h3>Step #6. Visualize the Predictions</h3>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
# Set plot size 
from pylab import rcParams
rcParams['figure.figsize'] = 18, 8

res = pd.merge(PREDICTIONS_FUTURE[target], dataset_test[target],left_index=True,right_index=True)
res.columns=['Predictions','Actual']
res['residuals'] = res['Predictions'] - res['Actual']

from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(res['Predictions'],res['Actual']))
print('rmse',rmse)
print('std test',np.std(res['Actual']))

rmse_str = str(round(rmse,2))


model_site_str='model_Gedi'
# Check whether the specified path exists or not
path = 'LSTM_models/'+model_site_str+'/mod1_rmse'+rmse_str
isExist = os.path.exists(path)
if not isExist:
  os.makedirs(path)
model.save(path)

function.plot_line(
  prediction=res.Predictions,
  actual=res.Actual,
  title=target_title+' Predcitions and Acutal Visitors at '+site_name,
  path_save='LSTM_models/'+model_site_str+'/',
  file_name='mod1_rmse'+ rmse_str+ '.png',
  fig_size_tuple=(20,5)

  )

function.plot_residuals(
  prediction=res.Predictions,
  actual=res.Actual,
  title='Residuals '+site_name,
  path_save='LSTM_models/'+model_site_str+'/',
  file_name='mod1_rmse'+ rmse_str+ 'residuals.png',
  fig_size_tuple=(20,5)
  )


In [ ]:
import lime
import lime.lime_tabular
# from lime.lime_text import RecurrentTabularExplainer
class_names=['Israelis_Count']

categorical_features = np.argwhere(
    np.array([len(set(dataset_train.values[:,x]))
    for x in range(dataset_train.values.shape[1])]) <= 10).flatten()
print(dataset_train.columns.shape)
print(categorical_features.shape)
print(X_train.shape)

explainer = lime.lime_tabular.RecurrentTabularExplainer(X_train, 
                                                   feature_names=dataset_train.columns,
                                                   class_names=['price'], 
                                                   categorical_features=categorical_features, 
                                                   verbose=True, mode='regression')


In [ ]:
exp = explainer.explain_instance(X_test[0],model.predict,num_features=10)
exp.show_in_notebook(show_table=True)


**What is the meaning of that?**


* left - we can see the 92th test value’s prediction
* middle - important (significant) features: directions and ranges for the prediction
* right - summary of results